In [176]:
# Import the different module
import urllib2, json
from bs4 import BeautifulSoup

In [177]:
# Main variable to scrap the website
WEBSITE_URL = "http://www.dailystormer.com/section/featured-stories/page/{0}/"
NUMBER_OF_PAGES = 200

# Build a list of seed URL to scrap
def build_seed_pages(url, number_of_pages_wanted):
    seeds = []
    for page_number in range(1, number_of_pages_wanted + 1):
        seeds.append(url.format(page_number))
    return seeds

# Given the seeds get the pages urls

def build_crawl_listing(seeds):
    url_listing = []
    for seed in seeds:
        opener = urllib2.build_opener()
        opener.addheaders = [('User-Agent', 'Mozilla/5.0')]
        response = opener.open(seed)
        soup = BeautifulSoup(response, 'html.parser')
        articles_list = soup.findAll("article", { "class" : "item-list" })
        for article in articles_list:
            title = article.find("h2")
            if title != -1:
                link = title.find('a')
                url_listing.append(link.get('href'))
    return url_listing

# Extract the content of some data from a given URL

def get_text_from_a_url(url):
    opener = urllib2.build_opener()
    opener.addheaders = [('User-Agent', 'Mozilla/5.0')]
    response = opener.open(url)
    soup = BeautifulSoup(response, 'html.parser')
    article = soup.find("div", { "class" : "entry" })
    blockquotes = article.findAll('blockquote') # Remove citation
    for blockquote in blockquotes: 
        blockquote.decompose()
    links = article.findAll('a') # Remove link
    for link in links:
        link.decompose()
    paragraphs = article.findAll("p")
    words_articles = ""
    for paragraph in paragraphs[1:]:
        words_articles = words_articles + ' ' + paragraph.text
    return words_articles

# Safe the data in a file

def safe_data(json_to_dump, file_title):
    output = json.dumps( json_to_dump, ensure_ascii=False, encoding='utf8')
    with open("data/{0}.json".format(file_title), "w+") as save:
        save.write(output.encode('utf8'))

In [178]:
# Build the crawl listing
seeds = build_seed_pages(WEBSITE_URL, NUMBER_OF_PAGES)
crawl_listing = build_crawl_listing(seeds)

# Build the list of article
list_of_article = []
for url in crawl_listing:
    list_of_article.append(get_text_from_a_url(url))

# Build a dictionnary URL : Text
result_from_crawl = {}
for index, url in enumerate(crawl_listing):
    result_from_crawl[url] = list_of_article[index]

# Safe the dictionnary
safe_data(result_from_crawl, 'articles')